In [46]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [47]:
import fasttext
import zipfile
import os
import zipfile
import os
import io
import re
import pandas as pd
import sklearn
import string
import re
import string
from scipy.stats import uniform
from sklearn import linear_model, datasets
from sklearn.model_selection import RandomizedSearchCV
import random

In [48]:
# vec_data = load_vectors(r"c:\Users\Saarthak\Desktop\detoxify_yt\model\wiki-news-300d-1M-subword.vec")
# model = fasttext.load_model(r"C:\Users\Saarthak\Desktop\detoxify_yt\model\wiki.en\wiki.en.bin")

In [49]:
# model.get_nearest_neighbors("chess")

In [50]:
#for kaggle : 
import kagglehub

# Download latest version
path = kagglehub.dataset_download("curlyoreki/detoxify")

print("Path to dataset files:", path)

df =  pd.read_csv("/kaggle/input/detoxify/df_final_dataset.csv")

# df = pd.read_csv(r"C:\Users\Saarthak\Desktop\detoxify_yt\dataset\df_final_dataset.csv")
# df.head()

c:\Users\Saarthak\Desktop\detoxify_yt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KaggleApiHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/curlyoreki/detoxify. The server reported the following issues: Permission 'datasets.get' was denied
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

In [39]:
def preprocess_text(text):
    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r"", text)

    # Remove punctuation and replace with spaces
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)

    # Remove non-English words
    words = text.split()
    english_words = []
    for word in words:
        try:
            if detect(word) == 'en':  # Check if the word is English
                english_words.append(word)
        except:
            continue  # Skip words that langdetect cannot classify
    text = " ".join(english_words)

    # Remove extra spaces and strip leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text.lower()

In [40]:
df['Title'] = df['Title'].map(preprocess_text)
df.head()

NameError: name 'df' is not defined

In [41]:
df['category_fasttext'] = '__label__' + df['category'].astype(str) + ' '
df = df[df['Title'].str.strip().astype(bool)].dropna(subset=['Title'])
df = df.reset_index(drop=True)
df['category_description'] = df['category_fasttext']+df['Title']
df.head(5)

NameError: name 'df' is not defined

In [42]:
from sklearn.model_selection import train_test_split
df_train  , val = train_test_split(df , test_size = 0.3)
train, test = train_test_split(df_train, test_size=0.2 )

NameError: name 'df' is not defined

In [43]:
train.head()

NameError: name 'train' is not defined

In [44]:
train.to_csv("fasttext_data.train", columns=["category_description"], index=False, header=False)
test.to_csv("fasttext_data.test", columns=["category_description"], index=False, header=False)
val.to_csv("fasttext_data.val", columns=["category_description"], index=False, header=False)

NameError: name 'train' is not defined

In [45]:
training = pd.read_csv("fasttext_data.train")
print(training)

     __label__coding medical coding basics abbrevations medicalcoding
0     __label__otherchelsea says goodbye to the cast...              
1     __label__otherprime minister boris johnson and...              
2     __label__coding the future of coding is finall...              
3     __label__coding is math really needed to code ...              
4              __label__othery a s official music video              
...                                                 ...              
1628  __label__math the man who solved the 1 million...              
1629  __label__otherthey had no idea how lucky they ...              
1630  __label__coding how to confuse a python developer              
1631  __label__math8 27 2020 lake charles la hurrica...              
1632  __label__otherterence crawford stops spence af...              

[1633 rows x 1 columns]


In [4]:
import fasttext
model = fasttext.train_supervised(input="/kaggle/input/fasttext/fasttext_data.train" , lr = 1e-4 , epoch = 25)

In [5]:
model.test("/kaggle/input/fasttext/fasttext_data.test")

(258, 0.3992248062015504, 0.3992248062015504)

In [13]:
#randomised grid search model on this

# Define the ranges for hyperparameters
hyperparameter_ranges = {
    'lr': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'lrUpdateRate': [10, 50, 100, 200],
    'epoch': [5, 25, 50, 100],
    'wordNgrams': [1, 2, 3],
    'dim': [50, 100, 200],
    'minCount': [1, 5, 10],
    'minCountLabel': [1, 5, 10],
    'bucket': [200000, 500000, 1000000],
    'ws': [3, 5, 10],
    'loss': ['ns', 'hs', 'softmax']
}

# Number of random combinations to try
n_iter = 35

best_precision = 0
best_recall = 0
best_params = {}

for i in range(n_iter):
    try:
        # Randomly sample hyperparameters
        params = {
            'lr': random.choice(hyperparameter_ranges['lr']),
            'lrUpdateRate': random.choice(hyperparameter_ranges['lrUpdateRate']),
            'epoch': random.choice(hyperparameter_ranges['epoch']),
            'wordNgrams': random.choice(hyperparameter_ranges['wordNgrams']),
            'dim': random.choice(hyperparameter_ranges['dim']),
            'minCount': random.choice(hyperparameter_ranges['minCount']),
            'minCountLabel': random.choice(hyperparameter_ranges['minCountLabel']),
            'bucket': random.choice(hyperparameter_ranges['bucket']),
            'ws': random.choice(hyperparameter_ranges['ws']),
            'loss': random.choice(hyperparameter_ranges['loss'])
        }
        
        # Train the model with the sampled hyperparameters
        model = fasttext.train_supervised(
            input="/kaggle/input/fasttext/fasttext_data.train",
            lr=params['lr'],
            lrUpdateRate=params['lrUpdateRate'],
            epoch=params['epoch'],
            wordNgrams=params['wordNgrams'],
            dim=params['dim'],
            minCount=params['minCount'],
            minCountLabel=params['minCountLabel'],
            bucket=params['bucket'],
            ws=params['ws'],
            loss=params['loss']
        )
        
        # Evaluate the model on the validation set
        result = model.test("/kaggle/input/fasttext/fasttext_data.val")
        precision = result[1]
        recall = result[2]
        
        # Check if this is the best model so far
        if precision + recall > best_precision + best_recall:
            best_precision = precision
            best_recall = recall
            best_params = params
        
        print(f"Iteration {i+1}/{n_iter}")
        print(f"Params: {params}")
        print(f"Precision: {precision}, Recall: {recall}")
        print("-----------------------------")
    except:
        print("error in : ")
        continue

# Print the best hyperparameters and corresponding precision and recall
print("Best Hyperparameters:")
print(best_params)
print(f"Best Precision: {best_precision}, Best Recall: {best_recall}")

# Train the final model with the best hyperparameters on the full training set

final_model = fasttext.train_supervised(
    input="/kaggle/input/fasttext/fasttext_data.train",
    lr=best_params['lr'],
    lrUpdateRate=best_params['lrUpdateRate'],
    epoch=best_params['epoch'],
    wordNgrams=best_params['wordNgrams'],
    dim=best_params['dim'],
    minCount=best_params['minCount'],
    minCountLabel=best_params['minCountLabel'],
    bucket=best_params['bucket'],
    ws=best_params['ws'],
    loss=best_params['loss']
)
# Evaluate the final model on the test set
final_result = final_model.test("/kaggle/input/fasttext/fasttext_data.test")
print(f"Final Model Precision: {final_result[1]}, Final Model Recall: {final_result[2]}")

Iteration 1/35
Params: {'lr': 0.1, 'lrUpdateRate': 100, 'epoch': 50, 'wordNgrams': 3, 'dim': 50, 'minCount': 10, 'minCountLabel': 10, 'bucket': 1000000, 'ws': 3, 'loss': 'softmax'}
Precision: 0.7857142857142857, Recall: 0.7857142857142857
-----------------------------
Iteration 2/35
Params: {'lr': 1e-05, 'lrUpdateRate': 200, 'epoch': 50, 'wordNgrams': 1, 'dim': 200, 'minCount': 1, 'minCountLabel': 5, 'bucket': 200000, 'ws': 10, 'loss': 'softmax'}
Precision: 0.6023255813953489, Recall: 0.6023255813953489
-----------------------------
error in : 
Iteration 4/35
Params: {'lr': 0.01, 'lrUpdateRate': 100, 'epoch': 25, 'wordNgrams': 2, 'dim': 200, 'minCount': 10, 'minCountLabel': 10, 'bucket': 500000, 'ws': 3, 'loss': 'hs'}
Precision: 0.5841836734693877, Recall: 0.5841836734693877
-----------------------------
Iteration 5/35
Params: {'lr': 0.0001, 'lrUpdateRate': 50, 'epoch': 100, 'wordNgrams': 2, 'dim': 200, 'minCount': 5, 'minCountLabel': 1, 'bucket': 200000, 'ws': 3, 'loss': 'hs'}
Precisi

In [16]:
#randomised grid search model on this

# Define the ranges for hyperparameters
hyperparameter_ranges = {
    'lr': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'lrUpdateRate': [10, 50, 100, 200],
    'epoch': [5, 25, 50, 100],
    'wordNgrams': [1, 2, 3],
    'dim': [50, 100, 200],
    'minCount': [1, 5, 10],
    'minCountLabel': [1, 5, 10],
    'bucket': [200000, 500000, 1000000],
    'ws': [3, 5, 10],
    'loss': ['ns', 'hs', 'softmax']
}


# Number of random combinations to try
n_iter = 1000

best_precision = 0
best_recall = 0
best_params = {}

for i in range(n_iter):
    try:
        # Randomly sample hyperparameters
        params = {
            'lr': random.choice(hyperparameter_ranges['lr']),
            'lrUpdateRate': random.choice(hyperparameter_ranges['lrUpdateRate']),
            'epoch': random.choice(hyperparameter_ranges['epoch']),
            'wordNgrams': random.choice(hyperparameter_ranges['wordNgrams']),
            'dim': random.choice(hyperparameter_ranges['dim']),
            'minCount': random.choice(hyperparameter_ranges['minCount']),
            'minCountLabel': random.choice(hyperparameter_ranges['minCountLabel']),
            'bucket': random.choice(hyperparameter_ranges['bucket']),
            'ws': random.choice(hyperparameter_ranges['ws']),
            'loss': random.choice(hyperparameter_ranges['loss'])
        }
        
        # Train the model with the sampled hyperparameters
        model = fasttext.train_supervised(
            input="/kaggle/input/fasttext/fasttext_data.train",
            lr=params['lr'],
            lrUpdateRate=params['lrUpdateRate'],
            epoch=params['epoch'],
            wordNgrams=params['wordNgrams'],
            dim=params['dim'],
            minCount=params['minCount'],
            minCountLabel=params['minCountLabel'],
            bucket=params['bucket'],
            ws=params['ws'],
            loss=params['loss']
        )
        
        # Evaluate the model on the validation set
        result = model.test("/kaggle/input/fasttext/fasttext_data.val")
        precision = result[1]
        recall = result[2]
        
        # Check if this is the best model so far
        if precision + recall > best_precision + best_recall:
            best_precision = precision
            best_recall = recall
            best_params = params
        
        print(f"Iteration {i+1}/{n_iter}")
        print(f"Params: {params}")
        print(f"Precision: {precision}, Recall: {recall}")
        print("-----------------------------")
    except:
        print("error in : ")
        continue

# Print the best hyperparameters and corresponding precision and recall
print("Best Hyperparameters:")
print(best_params)
print(f"Best Precision: {best_precision}, Best Recall: {best_recall}")

# Train the final model with the best hyperparameters on the full training set

final_model = fasttext.train_supervised(
    input="/kaggle/input/fasttext/fasttext_data.train",
    lr=best_params['lr'],
    lrUpdateRate=best_params['lrUpdateRate'],
    epoch=best_params['epoch'],
    wordNgrams=best_params['wordNgrams'],
    dim=best_params['dim'],
    minCount=best_params['minCount'],
    minCountLabel=best_params['minCountLabel'],
    bucket=best_params['bucket'],
    ws=best_params['ws'],
    loss=best_params['loss']
)
# Evaluate the final model on the test set
final_result = final_model.test("/kaggle/input/fasttext/fasttext_data.test")
print(f"Final Model Precision: {final_result[1]}, Final Model Recall: {final_result[2]}")

error in : 
error in : 
Iteration 3/1000
Params: {'lr': 0.0001, 'lrUpdateRate': 50, 'epoch': 50, 'wordNgrams': 2, 'dim': 100, 'minCount': 1, 'minCountLabel': 5, 'bucket': 500000, 'ws': 5, 'loss': 'ns'}
Precision: 0.09534883720930233, Recall: 0.09534883720930233
-----------------------------
Iteration 4/1000
Params: {'lr': 0.1, 'lrUpdateRate': 10, 'epoch': 25, 'wordNgrams': 3, 'dim': 100, 'minCount': 1, 'minCountLabel': 10, 'bucket': 200000, 'ws': 10, 'loss': 'hs'}
Precision: 0.6785714285714286, Recall: 0.6785714285714286
-----------------------------
error in : 
Iteration 6/1000
Params: {'lr': 0.0001, 'lrUpdateRate': 200, 'epoch': 5, 'wordNgrams': 2, 'dim': 200, 'minCount': 10, 'minCountLabel': 5, 'bucket': 200000, 'ws': 10, 'loss': 'ns'}
Precision: 0.002325581395348837, Recall: 0.002325581395348837
-----------------------------
Iteration 7/1000
Params: {'lr': 1e-05, 'lrUpdateRate': 10, 'epoch': 100, 'wordNgrams': 3, 'dim': 50, 'minCount': 5, 'minCountLabel': 10, 'bucket': 1000000, 'ws

In [17]:
final_model.save_model("final_model_fasttext.bin")

In [51]:
# import fasttext
# model = fasttext.load_model(r"C:\Users\Saarthak\Desktop\detoxify_yt\model\final_model_fasttext.bin")

In [1]:
print("hello world")

hello world


In [7]:
path = r"C:\Users\Saarthak\Desktop\detoxify_yt\model\detoxifying_fasttext\final_model_fasttext.bin"

In [11]:
import importlib
import fasttext
importlib.reload(fasttext)  #reloading the fasttext.py (did some changes)
model = fasttext.load_model(path)

In [13]:
# Test prediction
text = "10 coding habits to avoid"
label , prob = model.predict(text)

print(f"Label: {label}")
print(f"prob: {prob}") 

Label: ('__label__coding',)
prob: [0.99887925]


In [ ]:
# pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#uploading model to huggingface
import ipywidgets as widgets
from huggingface_hub import notebook_login

notebook_login()

model_save_path = r"C:\Users\Saarthak\Desktop\detoxify_yt\model\final_model_fasttext.bin"
# model.save_pretrained(model_save_path)